In [1]:
# a place to keep track of all corrections for each person accumulated through manual searches
# also merges in spreadsheet 'control_group_corrections.xlsx' with list of people who should not
# be in the control group and their imputed eod year

In [13]:
import pandas as pd
import numpy as np
import os
import funcy

from dev import CORRECTIONS_DIR, APP_DATA_DIR, NAME_COLS

In [53]:
# relevant filenames
dataset_filename = 'NIH_AAMC_index_cards_grant_standardized.csv'
control_corrections_filename = 'control_group_corrections.xlsx'

In [54]:
dataset_df = pd.read_csv(os.path.join(APP_DATA_DIR, dataset_filename))
corrections_df = pd.read_excel(os.path.join(CORRECTIONS_DIR, control_corrections_filename))

/home/lraymond/MIT/Azoulay_2016/yellow_berets/yb/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7,37,39,40,41,43,44,50,61,62,64,101,102) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [55]:
# get rid of excel blank lines
corrections_df = corrections_df[~pd.isnull(corrections_df['first_name'])]

In [63]:
corrections_df = corrections_df.rename(columns={
        'first_name': 'clean_first_name',
        'middle_name': 'clean_middle_name',
        'last_name': 'clean_last_name'})

In [64]:
corrections_df.head()

,clean_first_name,clean_middle_name,clean_last_name,application_year,directory_year,internship_year,residency_year,confirmed_NIH,imputed_directory_match,imputed_eod_year
0,BRETT,JOHN,GERSTENHABER,1971,1973,1971-1972,1972-1973,LIKELY,1,1973.0
1,PETER,LLOYD,GROSS,1970,1971-1972,1970-1971,1975,LIKELY,1,1971.0
2,ROBERT,STEPHEN,KATZ,1970,1972-1973,1970-1971,1971-1972,LIKELY,1,1972.0
3,BARRY,JOSHUA,KEMLER,1972,1975,1972,1973,LIKELY,1,1975.0
4,ROBERT,EDWARD,GERHARDT,1970,1971-1972,1970-1971,1971-1972,NO,0,NaN


In [66]:
print corrections_df.shape
dataset_df2 = pd.merge(
    left=dataset_df, right=corrections_df[
        NAME_COLS+['imputed_directory_match', 'imputed_eod_year']],
        on=NAME_COLS, how='left')
print dataset_df2.shape

(33, 10)
(4106, 151)


In [68]:
missing = corrections_df.loc[~corrections_df['clean_last_name'].isin(dataset_df2['clean_last_name'])]

In [69]:
missing.head()

,clean_first_name,clean_middle_name,clean_last_name,application_year,directory_year,internship_year,residency_year,confirmed_NIH,imputed_directory_match,imputed_eod_year


In [70]:
# for all people now excluded from the control, change control variable to 0 and update their eod year
imputed_treatment_mask = dataset_df2['imputed_directory_match']==1
dataset_df2.loc[imputed_treatment_mask, 'control_flag'] = 0
dataset_df2.loc[imputed_treatment_mask, 'eod_year'] =  dataset_df2.loc[imputed_treatment_mask, 'imputed_eod_year']


In [74]:
# other corrections

In [ ]:
# Brooks J Crawford should be J Brooks Crawford
